In [1]:
import pandas as pd
import numpy as np
#import extraction_squares
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os 
import sys

operating_system = 'mac'

if operating_system == 'win':
    os.chdir('C:/Users/fabau/OneDrive/Documents/GitHub/master-project-cleaned/')
elif operating_system == 'curnagl':
    os.chdir('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/')
    path = os.getcwd()
else:
    os.chdir('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/')

# Add the path to the custom library
custom_library_path = os.path.abspath('util/processing/')
sys.path.append(custom_library_path)

import extraction_squares, time_series#, pca

'''if operating_system == 'curnagl':
    name_of_variable= pd.read_csv('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/DATASETS/variable_list_80_mean.csv')
    path_data = '/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/DATASETS'
else:'''
variables= pd.read_csv('data/variable_list_levels.csv')
path_data = 'data'
path = os.getcwd()

'''storm_dates = pd.read_csv('pre_processing/tracks/storm_dates.csv')
path_tracks_1h_EU = 'pre_processing/tracks/ALL_TRACKS/tracks_1h_EU'
path_tracks_1h_non_EU = 'pre_processing/tracks/ALL_TRACKS/tracks_1h_non_EU'
dataset = 'datasets_1h_EU'
dataset_non_EU = 'datasets_1h_non_EU'''

"storm_dates = pd.read_csv('pre_processing/tracks/storm_dates.csv')\npath_tracks_1h_EU = 'pre_processing/tracks/ALL_TRACKS/tracks_1h_EU'\npath_tracks_1h_non_EU = 'pre_processing/tracks/ALL_TRACKS/tracks_1h_non_EU'\ndataset = 'datasets_1h_EU'\ndataset_non_EU = 'datasets_1h_non_EU"

In [2]:
def get_storms_data(data, storm_index):
        return data.loc[data['storm_index'].isin(storm_index)]

threshold = 0.98
seed_number = 42

#def to_pca_components(path, variables, seed_number, threshold=0.98):

# Get the storm indices from a non-EU dataset
x_storms = pd.read_csv(f'{path}/data/time_series_1h_non_EU/instantaneous_10m_wind_gust/instantaneous_10m_wind_gust_max.csv')
index_storm_EU = x_storms['storm_index'].copy()
# reduce by 1 to match the index of the storm in the EU dataset
index_storm_EU = index_storm_EU - 1

# Split the storm indices into training, test and validation based on the storm index
storm_index_training, storm_index_test, storm_index_validation = extraction_squares.split_storm_numbers(index_storm_EU, 0.15, seed_number)

# order the index of the storms
try:
    storm_index_training.sort_values()
    storm_index_test.sort_values()
    storm_index_validation.sort_values()
except:
    storm_index_training.sort()
    storm_index_test.sort()
    storm_index_validation.sort()

print("Storm indices for training:", storm_index_training)
print("Storm indices for test:", storm_index_test)
print("Storm indices for validation:", storm_index_validation)

# add 1 to the storm index to match the index of the storm in the timeseries dataset
storm_index_training = [x+1 for x in storm_index_training]
storm_index_test = [x+1 for x in storm_index_test]
storm_index_validation = [x+1 for x in storm_index_validation]

import pandas as pd

# Function to reshape variable data into long format
def reshape_variable_data(var_data, storm_index, variable_name, stat):
    """
    Reshape variable data into long format and filter for specific storm indices.

    Args:
        var_data (pd.DataFrame): Raw data.
        storm_index (list): List of storm indices to include.
        variable_name (str): Name of the variable.
        stat (str): Statistic type (mean, max, etc.).

    Returns:
        pd.DataFrame: Reshaped DataFrame with 'storm_number', 'step', and the variable.
    """
    # Filter data for specified storm indices
    var_data = var_data[var_data['storm_index'].isin(storm_index)]
    
    # Melt the DataFrame into long format
    var_data_long = var_data.melt(
        id_vars=['storm_index'],
        value_vars=[str(i) for i in range(36)],
        var_name='step',
        value_name=f"{variable_name}_{stat}"
    )

    # Add a storm_number column
    var_data_long['storm_number'] = 'storm_' + var_data_long['storm_index'].astype(str)

    # Reorganize columns
    var_data_long = var_data_long[['storm_number', 'step', f"{variable_name}_{stat}"]]

    return var_data_long


# Data is stored by statistics
stats = ['max', 'min', 'mean', 'std']

# Initialize empty DataFrames for the final combined sets
final_train = pd.DataFrame()
final_test = pd.DataFrame()
final_validation = pd.DataFrame()

# Load the data
for var in variables['variables']:
    if var == 'sea_surface_temperature':
        print('sea_surface_temperature is not taken into account')
        continue

    if var == 'instantaneous_10m_wind_gust':
        print('instantaneous_10m_wind_gust is not taken into account, for obvious reasons')
        continue

    # Process each statistic for the current variable
    for stat in stats:
        var_name = f"{var}_{stat}"
        var_data = pd.read_csv(f"{path}/data/time_series_1h_non_EU/{var}/{var}_{stat}.csv")

        # Reshape data for training, test, and validation sets
        var_training = reshape_variable_data(var_data, storm_index_training, var, stat)
        var_test = reshape_variable_data(var_data, storm_index_test, var, stat)
        var_validation = reshape_variable_data(var_data, storm_index_validation, var, stat)

        var_training['storm_number'] = var_training['storm_number'].str.replace('storm_', '').astype(int)
        var_test['storm_number'] = var_test['storm_number'].str.replace('storm_', '').astype(int)
        var_validation['storm_number'] = var_validation['storm_number'].str.replace('storm_', '').astype(int)

        # Merge the reshaped data into the final combined sets
        if final_train.empty:
            final_train = var_training
            final_test = var_test
            final_validation = var_validation
        else:
            final_train = final_train.merge(var_training, on=['storm_number', 'step'], how='outer')
            final_test = final_test.merge(var_test, on=['storm_number', 'step'], how='outer')
            final_validation = final_validation.merge(var_validation, on=['storm_number', 'step'], how='outer')

        var_all = pd.concat([var_training, var_test, var_validation]).reset_index(drop=True)

        var_training_pre_scaler = var_training.drop(columns=['storm_number', 'step'])
        var_test_pre_scaler = var_test.drop(columns=['storm_number', 'step'])
        var_validation_pre_scaler = var_validation.drop(columns=['storm_number', 'step'])

        var_all_pre_scaler = var_all.drop(columns=['storm_number', 'step'])

        # standardize the data by row
        #var_training = var_training.T
        scaler = StandardScaler()
        scaler.fit(var_training_pre_scaler)
        var_training_scaler = scaler.transform(var_training_pre_scaler)

        #var_test = var_test.T
        scaler = StandardScaler()
        scaler.fit(var_test_pre_scaler)
        var_test_scaler = scaler.transform(var_test_pre_scaler)

        #var_validation = var_validation.T
        scaler = StandardScaler()
        scaler.fit(var_validation_pre_scaler)
        var_validation_scaler = scaler.transform(var_validation_pre_scaler)

        #var_all = var_all.T
        scaler = StandardScaler()
        scaler.fit(var_all_pre_scaler)
        var_all_scaler = scaler.transform(var_all_pre_scaler)

        # add the storm number and step back to the data
        var_all_scaler = pd.DataFrame(var_all_scaler)
        var_all_scaler.rename(columns=lambda x: var_name, inplace=True)
        var_all_scaler['storm_number'] = var_all['storm_number']
        var_all_scaler['step'] = var_all['step']

        # add the storm number and step back to the data
        var_training_scaler = pd.DataFrame(var_training_scaler)
        var_training_scaler.rename(columns=lambda x: var_name, inplace=True)
        var_training_scaler['storm_number'] = var_training['storm_number']
        var_training_scaler['step'] = var_training['step']
        
        var_test_scaler = pd.DataFrame(var_test_scaler)
        var_test_scaler.rename(columns=lambda x: var_name, inplace=True)
        var_test_scaler['storm_number'] = var_test['storm_number']
        var_test_scaler['step'] = var_test['step']

        var_validation_scaler = pd.DataFrame(var_validation_scaler)
        var_validation_scaler.rename(columns=lambda x: var_name, inplace=True)
        var_validation_scaler['storm_number'] = var_validation['storm_number']
        var_validation_scaler['step'] = var_validation['step']
        

        # Train the PCA model
        pca = PCA()
        pca_test = PCA()
        pca_validation = PCA()

        # fit the PCA per storms
        pca.fit(var_training_scaler.T)
        explained_variance = pca.explained_variance_ratio_.cumsum()

        pca_test.fit(var_test_scaler)
        explained_variance_test = pca_test.explained_variance_ratio_.cumsum()

        pca_validation.fit(var_validation_scaler)
        explained_variance_validation = pca_validation.explained_variance_ratio_.cumsum()

        pca_all = PCA()
        pca_all.fit(var_all_scaler.drop(columns=['storm_number','step']).T)
        explained_variance_all = pca_all.explained_variance_ratio_.cumsum()

        break
        # get the number of components needed to explain x% of the variance
        explained_variance = pca.explained_variance_ratio_.cumsum()
        explained_variance_test = pca_test.explained_variance_ratio_.cumsum()
        explained_variance_validation = pca_validation.explained_variance_ratio_.cumsum()

        # Find the index of the first value meeting the threshold
        first_above_idx = np.argmax(explained_variance >= threshold)
        first_above_idx_test = np.argmax(explained_variance_test >= threshold)
        first_above_idx_validation = np.argmax(explained_variance_validation >= threshold)

        # Filter values
        n_components = explained_variance[(explained_variance < threshold) | (np.arange(len(explained_variance)) == first_above_idx)]
        n_components = explained_variance[:len(n_components)]

        n_components_test = explained_variance_test[(explained_variance_test < threshold) | (np.arange(len(explained_variance_test)) == first_above_idx_test)]
        n_components_test = explained_variance_test[:len(n_components_test)]

        n_components_validation = explained_variance_validation[(explained_variance_validation < threshold) | (np.arange(len(explained_variance_validation)) == first_above_idx_validation)]
        n_components_validation = explained_variance_validation[:len(n_components_validation)]

        print('We want to keep', threshold*100,'% of the variance, so we need', n_components.shape[0], 'components')

        # Transform the data
        scores = pca.transform(var_training_scaler)
        scores_threshold = scores[:, :n_components.shape[0]]

        scores_test = pca_test.transform(var_test_scaler)
        scores_threshold_test = scores_test[:, :n_components_test.shape[0]]

        scores_validation = pca_validation.transform(var_validation_scaler)
        scores_threshold_validation = scores_validation[:, :n_components_validation.shape[0]]

        # what we keep for the input of ML model is the scores_98
        scores_threshold = pd.DataFrame(scores_threshold)
        scores_threshold.rename(columns=lambda x: 'PCA_'+str(x+1), inplace=True)
        scores_threshold.to_csv(f'{path}/data/PCA_scores/training_{var_name}.csv')

        scores_threshold_test = pd.DataFrame(scores_threshold_test)
        scores_threshold_test.rename(columns=lambda x: 'PCA_'+str(x+1), inplace=True)
        scores_threshold_test.to_csv(f'{path}/data/PCA_scores/test_{var_name}.csv')

        scores_threshold_validation = pd.DataFrame(scores_threshold_validation)
        scores_threshold_validation.rename(columns=lambda x: 'PCA_'+str(x+1), inplace=True)
        scores_threshold_validation.to_csv(f'{path}/data/PCA_scores/validation_{var_name}.csv')

        print(f'PCA scores for {var_name} have been saved')

# Save the final combined datasets
final_train.to_csv(r'pre_processing\time_series_i10fg_before_eu/final_train.csv', index=False)
final_test.to_csv(r'pre_processing\time_series_i10fg_before_eu/final_test.csv', index=False)
final_validation.to_csv(r'pre_processing\time_series_i10fg_before_eu/final_validation.csv', index=False)

# merge all the data into one dataset
final_all = pd.concat([final_train, final_test, final_validation])
final_all.to_csv(r'pre_processing\time_series_i10fg_before_eu/final_all.csv', index=False)

print("Final datasets have been saved as 'final_train.csv', 'final_test.csv', and 'final_validation.csv'")

Storm indices for training: [0, 1, 2, 4, 6, 7, 10, 11, 12, 15, 18, 20, 25, 26, 30, 31, 33, 38, 43, 46, 48, 49, 50, 52, 53, 55, 57, 61, 62, 63, 64, 66, 67, 68, 71, 72, 75, 76, 78, 79, 80, 82, 84, 88, 89, 94]
Storm indices for test: [5, 28, 37, 44, 47, 65, 85, 86, 92]
Storm indices for validation: [32, 42, 45, 59, 60, 70, 77, 81]


c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site

instantaneous_10m_wind_gust is not taken into account, for obvious reasons


c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site

sea_surface_temperature is not taken into account


c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site-packages\sklearn\decomposition\_pca.py:557: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fabau\anaconda3\envs\tensor\lib\site

Final datasets have been saved as 'final_train.csv', 'final_test.csv', and 'final_validation.csv'


In [3]:
# Data is stored by statistics
stats = ['max', 'min', 'mean', 'std']

# Initialize empty DataFrames for the final combined sets
final_train = pd.DataFrame()
final_test = pd.DataFrame()
final_validation = pd.DataFrame()

all_pca = pd.DataFrame()

# Load the data
for var in variables['variables']:
    if var == 'sea_surface_temperature':
        print('sea_surface_temperature is not taken into account')
        continue

    if var == 'instantaneous_10m_wind_gust':
        print('instantaneous_10m_wind_gust is not taken into account, for obvious reasons')
        continue

    # Process each statistic for the current variable
    for stat in stats:
        var_name = f"{var}_{stat}"
        var_data = pd.read_csv(f"{path}/data/time_series_1h_non_EU/{var}/{var}_{stat}.csv")
        test_var_data = var_data.drop(columns=['Unnamed: 0'])

        # Threshold for variance retention
        threshold = 0.8  # 98% variance
        seed_number = 42

        # Assume var_all_scaler has columns: storm_number, step_1, step_2, ..., step_n

        # Step 1: Extract numerical data (exclude storm_number for scaling)
        storm_numbers = test_var_data['storm_index']
        data_to_scale = test_var_data.drop(columns=['storm_index'])

        # TO DISCUSS, standardize by storms per variable or just by variable ?

        # Step 2: Scale the entire dataset
        scaler = StandardScaler()
        #data_flattened = data_to_scale.values.flatten().reshape(-1, 1)
        data_scaled_flattened = scaler.fit_transform(data_to_scale)
        #data_scaled = data_scaled_flattened.reshape(data_to_scale.shape)
        data_scaled = data_scaled_flattened

        # Convert scaled data back to DataFrame with storm_number
        data_scaled_df = pd.DataFrame(data_scaled)#, columns=data_to_scale.columns)
        #data_scaled_df['storm_index'] = storm_numbers.values

        # Apply PCA
        pca = PCA(n_components=threshold, random_state=seed_number)
        data_pca = pca.fit_transform(data_scaled)

        # Calculate cumulative explained variance
        explained_variance = pca.explained_variance_ratio_.cumsum()

        # Find the number of components to explain 'threshold' variance
        #n_components_idx = np.argmax(explained_variance >= threshold) + 1  # +1 to convert to count
        n_components = len(explained_variance)
        print(f"To retain {threshold*100}% of the variance, we need {n_components} components.")

        # Transform the data to the reduced number of components
        #data_pca = pca.transform(data_scaled)[:, :n_components_idx]

        # Convert back to DataFrame
        data_pca_df = pd.DataFrame(data_pca, columns=[f'PCA_{i+1}' for i in range(n_components)])
        #data_pca_df = pd.DataFrame(data_pca)

        # Add storm_number back for reference if needed
        data_pca_df['storm_number'] = storm_numbers.values

        # put the storm number in the first column
        cols = data_pca_df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        data_pca_df = data_pca_df[cols]

        # add the variable name and stat to the data
        data_pca_df['variable'] = var_name

        # and set it as the second column
        cols = data_pca_df.columns.tolist()
        cols = cols[:1] + cols[-1:] + cols[1:-1]
        data_pca_df = data_pca_df[cols]
        
        # Append to results
        #all_storms_pca.append(storm_pca_df)

        # Step 4: Combine results into a single DataFrame
        #final_pca_df = pd.concat(all_storms_pca, axis=0)
        
        data_pca_df.to_csv(f'{path}/data/PCA_scores/{var_name}.csv', index=False)
        all_pca = pd.concat([all_pca, data_pca_df], axis=0)

        # Save the PCA loadings
        loadings = pd.DataFrame(pca.components_, columns=data_to_scale.columns)
        loadings.to_csv(f'{path}/data/PCA_loadings/{var_name}_loadings.csv', index=False)

# Save the final combined datasets
all_pca.to_csv(r'data/PCA_scores/all_pca.csv', index=False)
all_pca.to_csv(r'pre_processing/nestedMLR/all_pca.csv', index=False)

To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 2 components.
To retain 80.0% of the variance, we need 5 components.
To retain 

/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/decomposition/_pca.py:559: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/decomposition/_pca.py:559: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/decomposition/_pca.py:559: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/decomposition/_pca.py:559: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/decomposition/_pca.py

To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 

/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/decomposition/_pca.py:559: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/decomposition/_pca.py:559: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/decomposition/_pca.py:559: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/decomposition/_pca.py:559: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/sklearn/decomposition/_pca.py

To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 80.0% of the variance, we need 1 components.
To retain 

In [15]:
pca.explained_variance_ratio_.cumsum()

array([0.85397508, 0.95899359, 0.9783166 ])